In [58]:
import requests
import re
# Define the payload and headers for the request
# shorts channel: https://www.youtube.com/watch?v=xEuH_5Ik92U
# yt_shorts page: https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
payload = {
    'api_key': API_KEY,
    'url': 'https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts',
    # 'render': 'true'
}

headers = {
    'x-sapi-render': 'true',
    'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
    # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
    # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
}
# <div class="yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response"
# Make the GET request
r = requests.get('https://api.scraperapi.com/', params=payload,  verify=False) # headers=headers,
# Regular expression pattern to extract all instances of url paths like "/shorts/xxxx"
url_pattern = r'"url":"(/shorts/\w+)"'

# Extract all URL paths using regex
extracted_urls = re.findall(url_pattern, text)
# Save the response text to a file
with open('response_channel_shorts.txt', 'w', encoding='utf-8') as file:
    file.write(r.text)

print("Response saved to response_output.txt")


43


In [20]:
# scraperAPI result filter
def basic_shorts_info(text):
    #shorts title
    match_title = re.search(r'<title>(.*?) - YouTube</title>', r.text)
    if match_title:
        extracted_title = match_title.group(1)  # Get the content inside the <title>...</title> tags
    else:
        extracted_title = "No title found"
    # shorts description
    description_match = re.search(r'<meta name="description" content="(.*?)">', r.text)
    if description_match:
        extracted_description = description_match.group(1)  # Get the content inside the description meta tag
    else:
        extracted_description = "No description found"
    # shorts total view 
    date_time_pattern = r'datePublished" content="(\d{4}-\d{2}-\d{2})T(\d{2}):'
    # Find the date and time
    match_date = re.search(date_time_pattern, text)

    if match_date:
        extracted_datetime = f"{match_date.group(1)} {match_date.group(2)}"
    else:
        extracted_datetime = "No match found"

    # Thumbnail 
    thumbnail_pattern = r'"url":"(https?://[^\s"]+)"'
    # Extract the URL using regex
    thumbnail_match = re.search(thumbnail_pattern, text)
    if thumbnail_match:
        extracted_thumbnalil = thumbnail_match.group(1)
    else:
        extracted_thumbnalil = "No Thumbnail found"

    # Total view
    view_pattern = r'"viewCountEntityKey":"[^"]+","factoid":\{"factoidRenderer":\{"value":\{"simpleText":"([\d,]+)"'
    # Extract the number using regex only if viewCountEntityKey is present
    view_match = re.search(view_pattern, text)
    if view_match:
        extracted_view = view_match.group(1)
    else:
        extracted_view = "No number found"
        

    return extracted_title, extracted_description, extracted_datetime, extracted_thumbnalil, extracted_view

yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnalil, yt_view = basic_shorts_info(r.text)
print("yt_title: ", yt_title)
print("yt_shorts_description: ", yt_shorts_description)
print("yt_shorts_published: ", yt_shorts_published)
print("yt_shorts_thumbnalil: ", yt_shorts_thumbnalil)
print("yt_view: ", yt_view)
yt_shorts_script = ''


yt_title:  청각장애 팬을 감동시킨 스피드 ㅠ
yt_shorts_description:  #ishowspeed #shorts
yt_shorts_published:  2024-10-06 04
yt_shorts_thumbnalil:  https://i.ytimg.com/vi/xEuH_5Ik92U/hq2.jpg?sqp=-oaymwE1CKgBEF5IVfKriqkDKAgBFQAAiEIYAHABwAEG8AEB-AG2CIACgA-KAgwIABABGH8gGSgXMA8=\u0026rs=AOn4CLAr1jqDw2dLYSgK8pU5UBZfxZ7tPg
yt_view:  3,309,087


In [ ]:
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# List of URLs to test
urls = [
    'https://www.youtube.com/watch?v=xEuH_5Ik92U',
    'https://www.youtube.com/watch?v=CPsCEfs3m8I',
    'https://www.youtube.com/watch?v=jXUHFbNo5Vg'
]

# API key and headers
headers = {
    'x-sapi-render': 'true',
    # 'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
    'x-sapi-instruction_set': '[{"type":"loop","for":' + str(len(urls))+',"instructions":[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]}]'
}

# Loop through each URL, send request, and save response to a file
for i, url in enumerate(urls):
    payload = {
        'api_key': API_KEY,
        'url': url,
        'render': 'true'
    }
    
    # Send the request
    r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
    
    # Save the response to a file, naming it based on the URL index
    filename = f'response_{i + 1}.txt'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f'Response for {url} saved to {filename}')



### Daily 쇼츠 체크
- 95000 채널 매일 확인 KST 23:30 진행
- request 95000
- channel_df: YT 채널 데이터
- loading_df: 적제 쇼츠 데이터

In [ ]:
# 기존 channel list와 적재 데이터 loading
import requests
import pandas as pd
from datetime import datetime, timedelta
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# 한국, 미국 데이터 9507개
channel_df = pd.read_excel('us_korea_test.xlsx', sheet_name="us_kor", encoding='utf-8')
# 적재 DB: # -7d, -14d, -21d 만 필터링
loading_df = pd.read_excel("loading_db.xlsx",encoding='utf-8')
today = datetime.today()
loading_df['check_date'] = loading_df['check_date'].dt.date
filter_dates = [today.date() - timedelta(days=7), today.date() - timedelta(days=14), today.date() - timedelta(days=21)]
# Filter the dataframe based on check_date
loading_df = loading_df[loading_df['check_date'].isin(filter_dates)]
# channel_df 와 loading_df 교집합 데이터
merged_df = pd.merge(loading_df, channel_df, on='channel_name', how='inner')

##### 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d
1. 처음 돌릴때 최신 기준으로 1개 수집 
2. 기존 url 날짜에 맵핑이 되면 list에 추가

In [ ]:
import requests
from datetime import datetime, timedelta

# merge_df 후 각 채널 shorts url 확인 
def collect_past_shorts(api_key, merged_db):
    today = datetime.today()
    for check_dt, channel_shorts_url in zip(merged_db['check_date'], merged_db['yt_fixed_url']):
        if check_dt==(today.date() - timedelta(days=14)) | check_dt==(today.date() - timedelta(days=21)):
            payload = {
                'api_key': api_key,
                'url': channel_shorts_url
            }
            # Send the request
            r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
            shorts_url_pattern = r'"url":"(/shorts/\w+)"'

            # Extract all URL paths using regex
            extracted_urls = re.findall(shorts_url_pattern, r.text)

        # -7d에는 comments 추가 수집
        elif check_dt==(today.date() - timedelta(days=7)): 
            payload = {
                'api_key': API_KEY,
                'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U',
                'render': 'true'
            }

            headers = {
                'x-sapi-render': 'true',
                'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
                # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
                # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
            }



##### channel shorts url input으로 들어갈시 채널 url 필터링
1. 최신 기준으로 맵핑이 되지 않는 경우 최근 3개 url수집 
2. 기존 url 맵핑이 되면 list에 추가하여 return

In [52]:

# check_date: merged_df의 check_date
def scanning_preprocessing(channel_shorts_url, api_key, check_date):
    today = datetime.today()
    
    # -14d, -21d의 경우 좋아요, 커멘트수, view 수의 증가만 확인 진행
    if check_date==(today.date() - timedelta(days=14)) | check_date==(today.date() - timedelta(days=21)):
        payload = {
            'api_key': api_key,
            'url': channel_shorts_url
        }
        # Send the request
        r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
        shorts_url_pattern = r'"url":"(/shorts/\w+)"'

        # Extract all URL paths using regex
        extracted_urls = re.findall(shorts_url_pattern, r.text)
        # if extracted_urls loading_df
        # extracted_urls

    # -7d에는 comments 추가 수집
    elif check_date==(today.date() - timedelta(days=7)): 
        payload = {
            'api_key': API_KEY,
            'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U',
            'render': 'true'
        }

        headers = {
            'x-sapi-render': 'true',
            'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
            # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
            # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
        }





# 매일 전체 채널 필터링
# - 신규 쇼츠 데이터 발견시 list에 추가해서 comments까지 확인
# - 과거 데이터가 없을 수 있으니 만약 하나도 매칭이 안되면 3개 수집, 있다면 그거 앞에꺼를 추가수집
def scanning_yt_shorts(api_key, channel_db, loading_db):
    fixed_url_list = channel_db['yt_fixed_url'].to_list()
    channel_name_list = channel_db['channel_name'].to_list()

    # Loop through each URL, send request, and save response to a file
    for yt_nm, url in zip(channel_name_list, fixed_url_list):
        payload = {
            'api_key': api_key,
            'url': url
        }
        # Send the request
        r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
        # r.text는 YT 채널 쇼츠 페이지, 기존에 없던건 list로 저장, 있던 것중 shorts_published_date가 -7d, -14d, -21d에 매칭이 되는게 있다면 list에 추가


# scanning_yt_shorts(API_KEY, channel_df, loading_df)


In [51]:
# loading_db = pd.DataFrame(columns = ['channel_nm', 'shorts_url', 'shorts_title', 'shorts_description', 'shorts_published_date', 'shorts_thumbnalil', 'shorts_view', 'check_date'])
# loading_db.to_excel('loading_db.xlsx', index=False)